### Imports
Import useful modules and classes to perform the connection to MongoDB.

In [1]:
# Import os to manage paths
import os

# Import certifi to manage SSL certificates with TSL host
import certifi

# Import ConfigParser for handling config file
from configparser import ConfigParser

# Generate pretty outputs
from pprint import pprint

# For dates handling
from datetime import datetime

# Import PyMongo useful classes
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

### Read the configuration file and load the credentials
Load sensible data from a configuration file.

In [2]:
# Read the configuration file with the credentials
config = ConfigParser()
config.read(os.path.join(os.getcwd(), 'config.ini'))

# Load the data to connect to MongoDB Atlas Cluster
user = config['MongoDB']['user']
password = config['MongoDB']['password']
uri = config['MongoDB']['uri']
db_name = config['MongoDB']['db_name']

### Connect to MongoDB Atlas Cluster
Try to connect to Atlas Cluster (AWS Cloud) and print the server info to ensure connection. If the output is a dict, the connection is ok.

In [3]:
# Generate the connection string for the client
CONNECTION_STRING = "mongodb+srv://{}:{}@{}/{}?retryWrites=true&w=majority"\
                    .format(user, password, uri, db_name)

# Connect to MongoDB Atlas Cluster
client = MongoClient(CONNECTION_STRING, 
                     server_api=ServerApi('1'),
                     tls=True,
                     tlsCAFile=certifi.where())

# Check if connection succeed
try:
    pprint(client.server_info())
except Exception:
    print("Unable to connect to the server.")

{'$clusterTime': {'clusterTime': Timestamp(1652106897, 1),
                  'signature': {'hash': b'\x0e\xffQ\xfb\xfd\x887\t\xf0R\x18\xe4'
                                        b'{\x0f\x85\xa9\xb9\xb4\xf4\xf0',
                                'keyId': 7071822064142778372}},
 'allocator': 'tcmalloc',
 'bits': 64,
 'buildEnvironment': {'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0',
                      'ccflags': '-Werror -include mongo/platform/basic.h '
                                 '-fasynchronous-unwind-tables -ggdb -Wall '
                                 '-Wsign-compare -Wno-unknown-pragmas '
                                 '-Winvalid-pch -fno-omit-frame-pointer '
                                 '-fno-strict-aliasing -O2 -march=sandybridge '
                                 '-mtune=generic -mprefer-vector-width=128 '
                                 '-Wno-unused-local-typedefs '
                                 '-Wno-unused-function '
                          

### NoSQL vs SQL Databases
In order to compare NoSQL (MongoDB) to SQL Databases, let's see the following chart which link concepts between each other:

| Relational concept (SQL) | MongoDB equivalent (NoSQL) |
| :-: | :-: |
| Database | Database |
| Tables | Collections |
| Rows | Documents |
| Index | Index |

Once understood, we can continue the notebook retrieving some info from the Cloud. Let's see an example of some CRUD (Create, Read, Update, Delete) operations over a Sample Dataset that Atlas provides.

In [4]:
# Acces to the DB "sample_analytics"
# For more info, visit: https://www.mongodb.com/docs/atlas/sample-data/sample-analytics/
db = client.get_database('sample_analytics')

# Once pointed the DB, lest get all the collections names contained
db.list_collection_names()

['customers', 'accounts', 'transactions']

### CRUD Operations with PyMongo

Let's visualize some info of the *transactions* collection and make some CRUD operations.

This collection contains transactions details for users. Each document contains an account id, a count of how many transactions are in this set, the start and end dates for transactions covered by this document, and a list of sub documents. Each sub document represents a single transaction and the related information for that transaction.

For more info, please visit: https://www.mongodb.com/docs/atlas/sample-data/sample-analytics/#std-label-analytics-transactions

Below you can see the usage of the comparision operators.

In [5]:
# Create a new object with the collection (table)
transactions = db.get_collection('transactions')

print('Usage of the comparision operators of PyMongo:')

# Let's see how many documents (rows) are there in this collection (table)
num_docs = transactions.estimated_document_count()
print('There are {} documents in the transactions collection.'.format(num_docs))

# ¿How many documents with 6 transactions are in the DB?
trans_count = 6
six_trans_count = transactions.count_documents(filter={'transaction_count': trans_count})
print('There are {} documents in the transactions collection with {} transactions.'\
      .format(six_trans_count, trans_count))

# Other way to get the same result is as following:
six_trans_count = transactions.count_documents(filter={'transaction_count': {'$eq': trans_count}})
print('There are {} documents in the transactions collection with {} transactions.'\
      .format(six_trans_count, trans_count))

# Now let's see how many documents with more than 6 transactions are in the DB
more_six_trans_count = transactions.count_documents(filter={'transaction_count': {'$gt': trans_count}})
print('There are {} documents in the transactions collection with more than {} transactions.'\
      .format(more_six_trans_count, trans_count))

# ¿How many documents with 6 or 12 transactions are in the DB?
range_trans_count = transactions.count_documents(filter={'transaction_count': {'$in': [trans_count, 2*trans_count]}})
print('There are {} documents in the transactions collection with {} to {} transactions.'\
      .format(range_trans_count, trans_count, 2*trans_count))


Usage of the comparision operators of PyMongo:
There are 1746 documents in the transactions collection.
There are 13 documents in the transactions collection with 6 transactions.
There are 13 documents in the transactions collection with 6 transactions.
There are 1641 documents in the transactions collection with more than 6 transactions.
There are 33 documents in the transactions collection with 6 to 12 transactions.


In [6]:
# Visualize one of the examples
six_trans_docs = transactions.find_one(filter={'transaction_count': trans_count})
pprint(six_trans_docs)


{'_id': ObjectId('5ca4bbc1a2dd94ee58161cdf'),
 'account_id': 794875,
 'bucket_end_date': datetime.datetime(2016, 9, 6, 0, 0),
 'bucket_start_date': datetime.datetime(1991, 12, 27, 0, 0),
 'transaction_count': 6,
 'transactions': [{'amount': 1197,
                   'date': datetime.datetime(2011, 12, 28, 0, 0),
                   'price': '12.7330024299341033611199236474931240081787109375',
                   'symbol': 'nvda',
                   'total': '15241.40390863112172326054861',
                   'transaction_code': 'buy'},
                  {'amount': 8797,
                   'date': datetime.datetime(2016, 6, 13, 0, 0),
                   'price': '46.53873172406391489630550495348870754241943359375',
                   'symbol': 'nvda',
                   'total': '409401.2229765902593427995271',
                   'transaction_code': 'buy'},
                  {'amount': 6146,
                   'date': datetime.datetime(2016, 8, 31, 0, 0),
                   'price': '32.11

Now, let's see the logical operators

In [7]:
print('Example usage of the logical operators of PyMongo:')

start = datetime(1990, 1, 1, 0, 0, 0)

logical = transactions.find_one({
    '$and': [{
                'transaction_count': {'$gte': trans_count}
             },
             {
                'bucket_start_date': {'$gte': start}
             }]
})

# Visualize first result
pprint(logical)

Example usage of the logical operators of PyMongo:
{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb3'),
 'account_id': 557378,
 'bucket_end_date': datetime.datetime(2016, 11, 6, 0, 0),
 'bucket_start_date': datetime.datetime(1990, 6, 11, 0, 0),
 'transaction_count': 56,
 'transactions': [{'amount': 2561,
                   'date': datetime.datetime(2006, 10, 6, 0, 0),
                   'price': '38.236619210617988073863671161234378814697265625',
                   'symbol': 'adbe',
                   'total': '97923.98179839266745716486184',
                   'transaction_code': 'sell'},
                  {'amount': 9153,
                   'date': datetime.datetime(2000, 6, 19, 0, 0),
                   'price': '31.12236744839008650842515635304152965545654296875',
                   'symbol': 'adbe',
                   'total': '284863.0292551144618116154561',
                   'transaction_code': 'sell'},
                  {'amount': 18,
                   'date': datetime.datetime(2013